In [2]:
import timm
import torch
import torch.nn as nn

In [3]:
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

model = timm.create_model('vit_base_patch16_224', pretrained=True)

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

In [4]:
dc = model.named_children()
dc = list(dc)
x = torch.arange(4*3*224*224, dtype=torch.float32).reshape(4, 3, 224, 224)
x.shape

torch.Size([4, 3, 224, 224])

In [5]:
print(dc[0])
print("weight : ",dict(dc[0][1].named_children())['proj'].weight.reshape(768, 3, -1).shape)
print("\nx: ",x.shape)
print(dc[0][1](x).shape)

('patch_embed', PatchEmbed(
  (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (norm): Identity()
))
weight :  torch.Size([768, 3, 256])

x:  torch.Size([4, 3, 224, 224])
torch.Size([4, 196, 768])


In [6]:
14*14

196

In [7]:
dc[1]

('pos_drop', Dropout(p=0.0, inplace=False))

In [8]:
dc[2]

('patch_drop', Identity())

In [9]:
dc[3]

('norm_pre', Identity())

In [29]:
dd = dict(list(dc[4][1].children())[0].named_children())
qkv = dict(dd['attn'].named_children())['qkv']
print(qkv.weight.shape)
qkv_w = qkv.weight.clone().detach().numpy().T

torch.Size([2304, 768])


In [36]:
qkv_w.shape
qkv_w.tofile('./pre_weights/qkv_W.bin')

In [39]:
qkv_w.flatten()

array([ 0.00726264,  0.00374147, -0.07164224, ..., -0.00055039,
       -0.04176636, -0.00820988], dtype=float32)

In [ ]:
dd_ = list(dd.values())
x = torch.arange(4 * 196 *768, dtype=torch.float32).reshape(4, 196, 768)
x = dd_[0](x)
qkv = list(dd_[1].children())
print(x.shape)
x = qkv[0](x)
print(x.shape)
x = qkv[1](x)
print(qkv[1],x.shape)
x = qkv[2](x)
print(qkv[2],x.shape)
x = qkv[3](x)
print(qkv[3],x.shape)
# x = qkv[4](x)
print(qkv[4],x.shape)
print(dd_[1].forward)
model.forward

torch.Size([4, 196, 768])
torch.Size([4, 196, 2304])
Identity() torch.Size([4, 196, 2304])
Identity() torch.Size([4, 196, 2304])
Dropout(p=0.0, inplace=False) torch.Size([4, 196, 2304])
Linear(in_features=768, out_features=768, bias=True) torch.Size([4, 196, 2304])
<bound method Attention.forward of Attention(
  (qkv): Linear(in_features=768, out_features=2304, bias=True)
  (q_norm): Identity()
  (k_norm): Identity()
  (attn_drop): Dropout(p=0.0, inplace=False)
  (proj): Linear(in_features=768, out_features=768, bias=True)
  (proj_drop): Dropout(p=0.0, inplace=False)
)>


<bound method VisionTransformer.forward of VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.

In [ ]:
x = torch.arange(27,dtype=torch.float32).reshape(3, 3, 3)

for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        for k in range(x.shape[2]):
            x[i][j][k] = i

print(x)
nn.LayerNorm((3,3,3))(x)

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[2., 2., 2.],
         [2., 2., 2.],
         [2., 2., 2.]]])


tensor([[[-1.2247, -1.2247, -1.2247],
         [-1.2247, -1.2247, -1.2247],
         [-1.2247, -1.2247, -1.2247]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 1.2247,  1.2247,  1.2247],
         [ 1.2247,  1.2247,  1.2247],
         [ 1.2247,  1.2247,  1.2247]]], grad_fn=<NativeLayerNormBackward0>)